<a href="https://colab.research.google.com/github/Tati-AG/TP1_Mineracao_de_Dados/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import defaultdict
import pandas as pd
import numpy as np

## Importações

In [ ]:
tsv_file = '/content/drive/MyDrive/TP 1 - Mineracao de Dados/imdb_data/title.basics.tsv'

data_types = defaultdict (lambda: str, {5:'Int64', 6:'Int64'})

title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
                              converters = {7: lambda x: int(x) if x.isdigit() else pd.NA})

<ipython-input-3-609f254aab85>:5: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
<ipython-input-3-609f254aab85>:5: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
<ipython-input-3-609f254aab85>:5: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',


In [ ]:
tsv_file = '/content/drive/MyDrive/TP 1 - Mineracao de Dados/imdb_data/title.ratings.tsv'

data_types = {0:str, 1:'Float64', 2:'Int64'}

title_ratings = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
tsv_file = '/content/drive/MyDrive/TP 1 - Mineracao de Dados/imdb_data/title.crew.tsv'

data_types = defaultdict (lambda: str)

title_crew = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

In [ ]:
tsv_file = '/content/drive/MyDrive/TP 1 - Mineracao de Dados/imdb_data/name.basics.tsv'

data_types = defaultdict (lambda: str, {2:'Int64', 3:'Int64'})

name_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N')

## Exploração e Análise

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
sns.set()

## Limpeza e Seleção

In [ ]:
movie_basics = title_basics.loc[np.where ((title_basics['titleType'].values == 'movie'))]

In [ ]:
movie_ratings = title_ratings.loc[np.where ((title_ratings['numVotes'] >= 10000) & (title_ratings['tconst'].isin(movie_basics['tconst'])))]

In [ ]:
movie_ratings.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_ratings.csv', index = False)

In [ ]:
del title_basics
del title_ratings

In [ ]:
movie_basics = pd.merge (movie_basics, movie_ratings['tconst'], on = 'tconst', how = 'inner')
#pd.set_option('display.max_rows', None)
#print (movie_basics.groupby('startYear')['startYear'].count())

In [ ]:
genres_by_movie = movie_basics['genres'].str.split(',').explode()

In [ ]:
movie_genres = pd.DataFrame ({'genre' : genres_by_movie.unique()})

In [ ]:
movie_genre_bridge = pd.DataFrame ({'tconst' : movie_basics.loc[genres_by_movie.index]['tconst'].values,
                                    'genre' : genres_by_movie.values})

In [ ]:
movie_basics.drop('genres', axis='columns', inplace=True)
movie_basics.drop('isAdult', axis='columns', inplace=True)
movie_basics.drop('endYear', axis='columns', inplace=True)
movie_basics.drop('titleType', axis='columns', inplace=True)
del genres_by_movie

In [ ]:
movie_basics.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_basics.csv', index = False)
movie_genres.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_genres.csv', index = False)
movie_genre_bridge.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_genre_bridge.csv', index = False)

In [ ]:
movie_crew = title_crew.loc[np.where ((title_crew['tconst'].isin(movie_basics['tconst'])))]

In [ ]:
director_by_movie = movie_crew['directors'].str.split(',').explode()
movie_directors_bridge = pd.DataFrame ({'tconst' : movie_crew.loc[director_by_movie.index]['tconst'],
                                        'nconst' : director_by_movie.values}).dropna()

In [ ]:
movie_directors_bridge.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_director_bridge.csv', index = False)

In [ ]:
del title_crew
del movie_crew
del director_by_movie

In [ ]:
movie_directors = name_basics.loc[np.where ((name_basics['nconst'].isin(movie_directors_bridge['nconst'].unique())))]

In [ ]:
movie_directors.to_csv('/content/drive/My Drive/TP 1 - Mineracao de Dados/imdb_data/processed_data/movie_directors.csv', index = False)

In [ ]:
del name_basics

In [ ]:
print (('Movies Database: \t' + str (movie_basics.shape)).expandtabs(30))
print (('Movies x Genres Database: \t' + str (movie_genre_bridge.shape)).expandtabs(30))
print (('Genres Database: \t' + str (movie_genres.shape)).expandtabs(30))
print (('Movie Ratings Database: \t' + str (movie_ratings.shape)).expandtabs(30))
print (('Movies x Directors Database: \t' + str (movie_directors_bridge.shape)).expandtabs(30))
print (('Directors Database: \t' + str (movie_directors.shape)).expandtabs(30))

Movies Database:              (11439, 6)
Movies x Genres Database:     (29570, 2)
Genres Database:              (23, 1)
Movie Ratings Database:       (11439, 3)
Movies x Directors Database:  (12664, 2)
Directors Database:           (5169, 6)
